<a href="https://colab.research.google.com/github/Umar-Rauf09/BERT_Gender_Bias_Analysis./blob/main/Gender_%26_Profession_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Installing the Hugging Face "transformers" library (used for BERT)
!pip install transformers torch


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Loading a pre-trained English BERT model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
# Function: takes a word, returns its "embedding" (numeric meaning vector)
def get_embedding(word):
    tokens = tokenizer(word, return_tensors="pt")
    with torch.no_grad():                 # Don't train; just read BERT's knowledge
        outputs = model(**tokens)
    # Average all token embeddings into one vector for the word
    return outputs.last_hidden_state.mean(dim=1).numpy()


In [9]:
# Words representing gender
male_words = ["man", "he", "male"]
female_words = ["woman", "she", "female"]

# Profession words
professions = ["doctor", "nurse", "engineer", "Teacher", "chef", "writer", "artist"]


In [10]:
# Get average vectors for male and female groups
emb_male = np.mean([get_embedding(w) for w in male_words], axis=0)
emb_female = np.mean([get_embedding(w) for w in female_words], axis=0)

# Compare each profession with male and female
for job in professions:
    emb_job = get_embedding(job)
    sim_male = cosine_similarity(emb_job, emb_male)[0][0]
    sim_female = cosine_similarity(emb_job, emb_female)[0][0]

    # Decide which side it's closer to
    if sim_male > sim_female:
        bias = "Male-biased"
    elif sim_female > sim_male:
        bias = "Female-biased"
    else:
        bias = "Neutral"

    print(f"{job.capitalize():10s} → Male Sim: {sim_male:.3f} | Female Sim: {sim_female:.3f} | Bias: {bias}")


Doctor     → Male Sim: 0.881 | Female Sim: 0.870 | Bias: Male-biased
Nurse      → Male Sim: 0.853 | Female Sim: 0.861 | Bias: Female-biased
Engineer   → Male Sim: 0.735 | Female Sim: 0.745 | Bias: Female-biased
Teacher    → Male Sim: 0.818 | Female Sim: 0.830 | Bias: Female-biased
Chef       → Male Sim: 0.873 | Female Sim: 0.870 | Bias: Male-biased
Writer     → Male Sim: 0.834 | Female Sim: 0.846 | Bias: Female-biased
Artist     → Male Sim: 0.800 | Female Sim: 0.805 | Bias: Female-biased
